In [2]:
import numpy as np
import random

class DataFrame:

    def __init__(self, file_path):
        assert file_path is not None, 'Please inform the file path'
        self.__read_csv(file_path)
        
        '''dict for each column type by column'''
        self.__init_col_types()
        '''list for each column index by name'''
        self.__init_col_to_index_dict()        
        self.shape = (len(self.values), len(self.columns))

        self.factorized_cols = dict()

    def __getitem__(self, key):
        key_type = type(key)
        if key_type == int:
            return self.values[key]
        elif key_type == str:
            j = self.__get_col_index_by_key(key)
            return np.array(
                [self.values[i][j] for i in range(self.shape[0])],
                dtype=self.column_types[j]
            )
        elif key_type == tuple:
            assert len(key) == 2, 'Invalid tuple size'
            if key[1] < 0:
                key = (key[0], self.shape[1] - key[1])
            new_values = []
            for row in self.values:
                new_values.append([x for i, x in enumerate(row) if i >= key[0] and i < key[1]])
            return new_values
        else:
            raise Exception('Invalid key type')

    def __init_col_to_index_dict(self):
        self.colum_to_index_dict = dict()
        for i, c in enumerate(self.columns):
            self.colum_to_index_dict[c] = i

    def __init_col_types(self):
        def get_value_type(v):
            try:
                int(v)
                return int
            except Exception:
                pass

            try:
                float(v)
                return float
            except Exception:
                pass

            return str

        self.column_types = []
        for j in range(len(self.columns)):
            type = get_value_type(self.values[0][j])
            self.column_types.append(type)
            for i in range(len(self.values)):
                self.values[i][j] = type(self.values[i][j])

    def __read_csv(self, file_path):
        with open(file_path, 'r') as f:
            lines = f.readlines()
        
        self.columns = lines[0].replace('\n', '').split(',')
        self.values = [l.replace('\n', '').split(',') for l in lines[1:]]

    def __get_col_index_by_key(self, key):
        key_type = type(key)
        if key_type == str:
            assert key in self.columns, 'Invalid key'
            return self.colum_to_index_dict[key]
        elif key_type == int:
            return key
        else:
            raise Exception('Invalid key type')

    def __continuous_split(self, key):
            col_i = self.__get_col_index_by_key(key)
            assert self.column_types[col_i] == float, 'This can only be done to continuous valued columns'

            values = self[self.columns[col_i]]
            median = np.median(values)
            median_str = '{:.2f}'.format(median)
        
            for i in range(self.shape[0]):
                self.values[i][col_i] = f'<={median_str}' if self.values[i][col_i] <= median else f'>{median_str}'
            self.column_types[col_i] = str

    def categorize_continuous_values(self):
        for i, t in enumerate(self.column_types):
            if t == float:
                self.__continuous_split(i)

    def factorize(self, key):
        col_i = self.__get_col_index_by_key(key)

        classes = set(self[self.columns[col_i]])
        class_dict = dict()
        inverse_class_dict = dict()
        for i, c in enumerate(classes):
            class_dict[c] = i
            inverse_class_dict[i] = c
        self.factorized_cols[self.columns[col_i]] = inverse_class_dict

        for i in range(self.shape[0]):
            self.values[i][col_i] = class_dict[self.values[i][col_i]]
        self.column_types[col_i] = int

    def train_test_split(self, X_indexes, y_class, perc=.33):
        split_index = int(self.shape[0] * (1 - perc))
        shuffled_data = self.values
        random.shuffle(shuffled_data)
        
        train, test = shuffled_data[:split_index], shuffled_data[split_index:]


df = DataFrame(file_path='datasets/iris.csv')
df.factorize('class')
df.categorize_continuous_values()

print(df.columns)
print(df.values[:5])

['ID', 'sepallength', 'sepalwidth', 'petallength', 'petalwidth', 'class']
[[1, '<=5.80', '>3.00', '<=4.35', '<=1.30', 2], [2, '<=5.80', '<=3.00', '<=4.35', '<=1.30', 2], [3, '<=5.80', '>3.00', '<=4.35', '<=1.30', 2], [4, '<=5.80', '>3.00', '<=4.35', '<=1.30', 2], [5, '<=5.80', '>3.00', '<=4.35', '<=1.30', 2]]


In [27]:
class Node:

    def __init__(self, X, y, columns, parent=None, original_col_indexes=None):
        if parent is None:
            assert len(X) > 0 and len(y) > 0
        else:
            self.parent = parent

        assert len(X) == len(y)
        self.X, self.y = X, y
        self.parent = parent
        self.columns = columns
        
        if original_col_indexes is not None:
            self.original_col_indexes = original_col_indexes
        else:
            self.original_col_indexes = list(range(len(columns)))

        self.is_leaf = self.__decide_if_leaf()
        if not self.is_leaf:
            self.children = []
            self.class_to_children = dict()
            self.generate_children()

    def __call__(self, x):
        if self.is_leaf:
            return self.leaf_value
        else:
            return self.children[self.class_to_children[x[self.deciding_col]]](x)

    def __get_column_values(self, col):
        #assert type(col) != int, f'Invalid key type: {type(col)}'
        assert col >= 0 and col < len(self.columns), 'Invalid key value'
        return [self.X[i][col] for i in range(len(self.X))]

    def __decide_if_leaf(self):
        if len(self.y) == 0:
            self.leaf_value = self.parent.most_common_y()
            return True
        elif all([i == self.y[0] for i in self.y]):
            self.leaf_value = self.y[0]
            return True
        elif len(self.columns) == 0:
            self.leaf_value = self.parent.most_common_y()
            return True
        
        return False

    def __class_entropy(self, col_values):
        value_counter = dict()
        for v in col_values:
            if v not in value_counter.keys():
                value_counter[v] = 1
            else:
                value_counter[v] += 1

        col_size = len(col_values)
        for k in value_counter.keys():
            value_counter[k] /= col_size
        
        return sum([-p * np.log2(p) for _, p in value_counter.items()])

    def __drop_column(self, X, col):
        new_X = []
        for x in X:
            new_X.append([elem for i, elem in enumerate(x) if i != col])
        return new_X

    def __local_index_of_original(self, target):
        for i, original_i in enumerate(self.original_col_indexes):
            if original_i == target:
                return i
        raise Exception('Index not found')

    def most_common_y(self):
        biggest_count = -1
        most_common_y = -1
        for y, count in np.unique(self.y, return_counts=True):
            if count > biggest_count:
                biggest_count = count
                most_common_y = y
        return most_common_y 
    
    def entropy(self, column):
        col_values = self.__get_column_values(column)
        return self.__class_entropy(col_values)

    def generate_children(self):
        max_entropy_col = self.original_col_indexes[0]
        max_entropy = self.entropy(0)
        for i in range(len(self.columns)):
            tmp_entropy = self.entropy(i)
            if tmp_entropy > max_entropy:
                max_entropy = tmp_entropy
                max_entropy_col = self.original_col_indexes[i]
        
        self.deciding_col = max_entropy_col
        local_max_entropy_index = self.__local_index_of_original(max_entropy_col)
        
        new_cols, new_original_col_indexes = [], []
        for c, original_i in zip(self.columns, self.original_col_indexes):
            if original_i != max_entropy_col:
                new_cols.append(c)
                new_original_col_indexes.append(original_i)
        #new_cols, new_original_col_indexes = zip(*[(c, original_i) for c, original_i in zip(self.columns, self.original_col_indexes) if original_i != max_entropy_col])

        classes = np.unique(self.__get_column_values(local_max_entropy_index))
        for i, c in enumerate(classes):
            new_X, new_y = [], []
            for x, y in zip(self.X, self.y):
                if x[local_max_entropy_index] == c:
                    new_X.append(x)
                    new_y.append(y)
            #new_dataset = [(x, y) for x, y in zip(self.X, self.y) if x[local_max_entropy_index] == c]
    
            #new_X, new_y = zip(*new_dataset)
            new_X = self.__drop_column(new_X, local_max_entropy_index)

            self.class_to_children[c] = i
            self.children.append(Node(new_X, new_y, new_cols, self, new_original_col_indexes))
            
X = df[(1, -1)]
y = df['class']
cols = df.columns[1:-1]

root = Node(X, y, cols)
root(X[3])

2

In [32]:
class DecisionTree:

    def __init__(self, X, y, columns, inverse_class_dict):
        self.inverse_class_dict = inverse_class_dict
        self.root = Node(X, y, columns)

    def __call__(self, x):
        return self.predict(x)

    def predict(self, x):
        prediction = self.root(x)
        return self.inverse_class_dict[prediction]

dt = DecisionTree(X, y, cols, df.factorized_cols['class'])
dt(X[0]) 

'Iris-setosa'